In [ ]:
%pylab inline
%matplotlib notebook

from scipy.sparse.linalg import spsolve
from scipy.sparse import spdiags, tril, diags
import time

In [ ]:
def utrue(x):
    return sin(2*pi*x)

def rhs(x):
    return -(2*pi)**2*sin(2*pi*x)

In [ ]:
# Jacobi iteration
def jacobi(A,F,kmax,tol):
    D = diags(A.diagonal(),format='csc')   # Specify format to avoid warning message
    uk = zeros(F.shape) 
    rk = F-A@uk
    zk = spsolve(D,rk)
    zk = reshape(zk,F.shape)
    e = []
    for k in range(0,kmax):
        ukp1 = uk + zk
        e = e + [norm(zk)]        
        # print("{:5d} {:20.16e} {:20.16e}".format(k,e[k],norm(uk-ukp1)))            
        if e[k] < tol:
            break
        rk = F-A@ukp1
        zk = spsolve(D,rk)
        zk = reshape(zk,F.shape)
        uk = ukp1
    
    return ukp1,e,k



In [ ]:
# Gauss-Seidel
def gauss_seidel(A,F,kmax,tol):
    L = tril(A,format='csc')   # Specify format to avoid warning message
    uk = zeros(F.shape) 
    rk = F-A@uk
    zk = spsolve(L,rk)
    zk = reshape(zk,F.shape)
    e = []
    for k in range(0,kmax):
        ukp1 = uk + zk
        e = e + [norm(zk)]        
        # print("{:5d} {:20.16e} {:20.16e}".format(k,e[k],norm(uk-ukp1)))            
        if e[k] < tol:
            break
        rk = F-A@ukp1
        zk = spsolve(L,rk)
        zk = reshape(zk,F.shape)
        uk = ukp1
    
    return ukp1,e,k        

In [ ]:
N = 80
h = 1/N
xlong = linspace(0,1,N+1)
x = reshape(xlong[1:-1],(N-1,1))  # remove entries 0 and N+1; reshape vector.

z = ones(N-1)
A = spdiags([z,-2*z,z],[-1,0,1],N-1,N-1,format='csc')  # sparse storage
    
# Boundary conditions    
bc = zeros((N-1,1))    # "tuple" (N-1,1) describes shape of the array.
    
F = h**2*rhs(x) - bc

kmax = 10000
tol = 1e-12

t0 = time.time()
u,e_jacobi,itcount = jacobi(A,F,kmax,tol)
t1 = time.time()
print("{:>20s} {:>12s}".format("Method","Jacobi"))
print("{:>20s} {:12.4e}".format("Residual",e_jacobi[-1]))
print("{:>20s} {:12.4e}".format("Error",norm(u-utrue(x))))
print("{:>20s} {:12d}".format("# Iterations",itcount))
print("{:>20s} {:12.4f}".format("Time (s)",t1-t0))

print("")
t0 = time.time()
u,e_gs,itcount = gauss_seidel(A,F,kmax,tol)
t1 = time.time()
print("{:>20s} {:>12s}".format("Method","Gauss-Seidel"))
print("{:>20s} {:12.4e}".format("Residual",e_gs[-1]))
print("{:>20s} {:12.4e}".format("Error",norm(u-utrue(x))))
print("{:>20s} {:12d}".format("# Iterations",itcount))
print("{:>20s} {:12.4f}".format("Time (s)",t1-t0))


semilogy(e_jacobi,'r.-',markersize=5,label='Jacobi')
semilogy(e_gs,'b.-',markersize=5,label='Gauss-Seidel')

legend()